In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tslearn.preprocessing import TimeSeriesResampler
import random
from typing import List, Tuple

from oscillogram_classification.cam import gen_heatmap_dictionary, plot_heatmaps_as_overlay

In [ ]:
def load_signals(path: str) -> Tuple[List[int], List[List[float]]]:
    # dataframe containing all signals from the dataset + labels in col 0
    df = pd.read_csv(path, delimiter='\t', header=None, na_values=['-∞', '∞'])
    labels = df.iloc[:, 0].tolist()
    samples = df.iloc[:, 1:].values.tolist()
    return labels, samples

def z_normalize_time_series(series: List[float]) -> np.ndarray:
    return (series - np.mean(series)) / np.std(series)

def plot_signals(signals: np.ndarray, figsize: Tuple[int, int]) -> None:
    fig, axs = plt.subplots(len(signals), figsize=figsize)
    for signal_idx, signal in enumerate(signals):
        axs[signal_idx].plot(signal)
        axs[signal_idx].set_title("sample " + str(signal_idx))
    plt.tight_layout()
    plt.savefig("data_vis.svg", format="svg", bbox_inches='tight')
    plt.show()
    
def resample(signals: np.ndarray, znorm: bool) -> np.ndarray:
    target_len = 1250
    print("target len", target_len)
    for i in range(len(signals)):
        if len(signals[i]) != target_len:
            sig_arr = np.array(signals[i])
            sig_arr = sig_arr.reshape((1, len(signals[i]), 1))  # n_ts, sz, d
            signals[i] = TimeSeriesResampler(sz=target_len).fit_transform(sig_arr).tolist()[0]
        # z-normalization
        if znorm:
            signals[i] = z_normalize_time_series(signals[i])
    return np.array(signals)

In [ ]:
# creating a binary classification dataset for EOGVerticalSignal

def save_signals(path: str, labels: List[int], samples: List[List[float]]):
    df = pd.DataFrame(samples)
    df.insert(0, "label", labels)  # insert labels as first col
    df.to_csv(path, sep='\t', header=False, index=False)

train_data = "EOGVerticalSignal/EOGVerticalSignal_TRAIN.tsv"
train_labels, train_samples = load_signals(train_data)
train_labels = [i - 1 for i in train_labels]
# adjust the labels -- turn into binary classification problem
#   - subsume 0-10 as regular (0) and 11 as anomaly (1)
train_labels = [0 if label <= 10 else 1 for label in train_labels]
save_signals('EOGVerticalSignal/EOGVerticalSignal_TRAIN_BINARY.tsv', train_labels, train_samples)

test_data = "EOGVerticalSignal/EOGVerticalSignal_TEST.tsv"
test_labels, test_samples = load_signals(test_data)
test_labels = [i - 1 for i in test_labels]
# adjust the labels -- turn into binary classification problem
#   - subsume 0-10 as regular (0) and 11 as anomaly (1)
test_labels = [0 if label <= 10 else 1 for label in test_labels]
save_signals('EOGVerticalSignal/EOGVerticalSignal_TEST_BINARY.tsv', test_labels, test_samples)

In [ ]:
import os

# train_data = "Lightning7/Lightning7_TRAIN.tsv"
# test_data = "Lightning7/Lightning7_TEST.tsv"

#train_data = "Plane/Plane_TRAIN.tsv"
#test_data = "Plane/Plane_TEST.tsv"

#train_data = "MelbournePedestrian/MelbournePedestrian_TRAIN.tsv"
#test_data = "MelbournePedestrian/MelbournePedestrian_TEST.tsv"

#train_data = "SemgHandSubjectCh2/SemgHandSubjectCh2_TRAIN.tsv"
#test_data = "SemgHandSubjectCh2/SemgHandSubjectCh2_TEST.tsv"

#train_data = "EthanolLevel/EthanolLevel_TRAIN.tsv"
#test_data = "EthanolLevel/EthanolLevel_TEST.tsv"

train_data = "EOGVerticalSignal/EOGVerticalSignal_TRAIN.tsv"
test_data = "EOGVerticalSignal/EOGVerticalSignal_TEST.tsv"

train_labels, train_signals = load_signals(train_data)
print("#samples (train):", len(train_signals))
print("sample len:", len(train_signals[0]))
print("#classes (train):", len(np.unique(train_labels)))

test_labels, test_signals = load_signals(test_data)
print("\n#samples (test):", len(test_signals))
print("sample len:", len(test_signals[0]))
print("#classes (test):", len(np.unique(test_labels)))

train_labels = [i - 1 for i in train_labels]
test_labels = [i - 1 for i in test_labels]

print(np.unique(train_labels))

print(
    "train\tclass 0:", train_labels.count(0),
    "\tclass 1:", train_labels.count(1),
    "\tclass 2:", train_labels.count(2),
    "\tclass 3:", train_labels.count(3),
    "\tclass 4:", train_labels.count(4),
    "\tclass 5:", train_labels.count(5),
    "\tclass 6:", train_labels.count(6),
)
print(
    "test\tclass 0:", test_labels.count(0),
    "\tclass 1:", test_labels.count(1),
    "\tclass 2:", test_labels.count(2),
    "\tclass 3:", test_labels.count(3),
    "\tclass 4:", test_labels.count(4),
    "\tclass 5:", test_labels.count(5),
    "\tclass 6:", test_labels.count(6),
)

In [ ]:
# downsampling - True -> znorm
train_signals = resample(train_signals, True)
test_signals = resample(test_signals, True)

In [ ]:
# raw TS
train_signals = np.array(train_signals)
test_signals = np.array(test_signals)

In [ ]:
# vis - one sample for each class

unique_labels = np.unique(train_labels)

one_sample_each = []
for label in unique_labels:
    for i in range(len(train_labels)):
        if train_labels[i] == label:
            one_sample_each.append(train_signals[i])
            break

plot_signals(one_sample_each, figsize=(10, len(one_sample_each)))

In [ ]:
# vis - all samples for one class

label = 1
all_of_class = []
for i in range(len(train_labels)):
    if train_labels[i] == label:
        all_of_class.append(train_signals[i])

plot_signals(all_of_class, figsize=(10, len(all_of_class)))

## Training with z-normalized data

In [ ]:
import keras

# deactivate tensorflow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

print("before:", train_signals.shape)
print("before:", test_signals.shape)

num_samples = train_signals.shape[0]
sample_len = train_signals.shape[1]

In [ ]:
# univariate -- only if raW
print(train_signals.shape)

train_signals = train_signals[:, :, np.newaxis]
test_signals = test_signals[:, :, np.newaxis]

plt.plot(train_signals[0])
plt.title("First Sig")
plt.show()

In [ ]:
print("after:", train_signals.shape)
print("after:", test_signals.shape)

num_classes = len(np.unique(train_labels))
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Build model

- FCN
- hyperparameters (`kernel_size, filters, usage of BatchNorm`) found using `KerasTuner`

In [ ]:
def build_model(input_shape: np.ndarray) -> keras.models.Model:
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = build_model(input_shape=train_signals.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

- predefined ResNet

In [ ]:
from oscillogram_classification import models

model = models.create_resnet_model(input_shape=train_signals.shape[1:], num_classes=num_classes)
keras.utils.plot_model(model, show_shapes=True)

## Train model

In [ ]:
# there should be no model, otherwise retraining!
assert not os.path.isfile("best_model.keras")

epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1)
]

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

history = model.fit(
    train_signals,
    train_labels,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
# eval model on test data

model = keras.models.load_model("best_model.keras")
test_loss, test_acc = model.evaluate(test_signals, test_labels)

print("test acc.:", test_acc)
print("test loss:", test_loss)

In [ ]:
# plot training and validation loss

metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

### GradCAM++ on univariate data

In [ ]:
method = "tf-keras-gradcam++"

random_index = random.randint(0, len(test_signals) - 1)
net_input = test_signals[random_index]
assert net_input.shape[1] == 1
ground_truth = test_labels[random_index]
prediction = model.predict(np.array([net_input]))
heatmaps = gen_heatmap_dictionary(method, np.array(net_input), model, prediction)

In [ ]:
plot_heatmaps_as_overlay(heatmaps, net_input, 'test_plot', test_time_values.squeeze()[random_index].tolist())

## tsai training

In [ ]:
from tsai.all import *
import sklearn.metrics as skm
my_setup()

In [ ]:
def generate_tsai_dataset(train_signals: np.ndarray, train_labels: np.ndarray) -> TSDataLoaders:
    # randomly split the indices of the training samples into two sets (train (80%) and validation (20%))
    # 'splits' contains a tuple of lists ([train_indices], [validation_indices])
    #    - stratify=True -> split the data in such a way that each class's proportion in the train and validation
    #      datasets is approximately the same as the proportion in the original dataset
    #    - random_state is the seed
    splits = get_splits(train_labels, valid_size=.2, stratify=True, random_state=23, shuffle=True)
    print(splits)
    print("--> currently, the above plot wrongly labels 'Valid' as 'Test'")

    # define transformations:
    #    - None -> no transformation to the input (X)
    #    - Categorize() -> convert labels into categorical format; converts the labels to integers
    # my labels are already ints, but I'll leave it here as a more general case
    tfms  = [None, [Categorize()]]

    # creates tensors to train on, e.g.,
    #    dsets[0]: (TSTensor(vars:5, len:500, device=cpu, dtype=torch.float32), TensorCategory(0))
    dsets = TSDatasets(train_signals, train_labels, tfms=tfms, splits=splits, inplace=True)

    print("#train samples:", len(dsets.train))
    print("#valid samples:", len(dsets.valid))

    # data loaders: loading data in batches; batch size 64 for training and 128 for validation
    #    - TSStandardize: batch normalization
    #    - num_workers: 0 -> data loaded in main process
    dls = TSDataLoaders.from_dsets(
        dsets.train, dsets.valid, bs=[16, 32], batch_tfms=[TSStandardize()], num_workers=0
    )
    # vis a batch
    dls.show_batch(nrows=3, ncols=3, sharey=True)
    return dls

def train_tsai_model(dls: TSDataLoaders, model: XCM) -> Learner:
    # learner encapsulates the data, the model, and other details related to the training process
    learn = Learner(dls, model, metrics=accuracy, loss_func=CrossEntropyLossFlat())

    # saves curr state of learner (model + weights) to a file named stage0
    learn.save('stage0')

    # load state of model
    learn.load('stage0')

    # training over range of learning rates -- find suitable LR (or LR range)
    #    - learning rate range where the loss decreases most effectively
    learn.lr_find()

    # 150 -> num of epochs
    #    - involves varying the learning rate in a specific way during training
    #    - the cyclical nature helps in faster convergence, avoids getting stuck in local minima,
    #      and sometimes achieves better overall performance
    #    - it provides a balance between exploring the loss landscape (with higher learning rates)
    #    - and exploiting known good areas of the landscape (with lower learning rates)
    learn.fit_one_cycle(300, lr_max=1e-3)

    learn.save('stage1')
    return learn

def test_tsai_model(test_signals: np.ndarray, test_labels: np.ndarray, learn: Learner) -> np.float64:
    # labeled test data
    test_ds = TSDatasets(test_signals, test_labels, tfms=[None, [Categorize()]])
    test_dl = dls.valid.new(test_ds)

    test_probas, test_targets, test_preds = learn.get_preds(
        dl=test_dl, with_decoded=True, save_preds=None, save_targs=None
    )    
    return skm.accuracy_score(test_targets, test_preds)

In [ ]:
# tsai expects the data in a diff format: (samples, variables, length)

# variables = 1 for univariate datasets and >1 for multivariate

train_signals = train_signals.reshape(train_signals.shape[0], 1, train_signals.shape[1])
test_signals = test_signals.reshape(test_signals.shape[0], 1, test_signals.shape[1])

print(train_signals.shape)
print(test_signals.shape)

In [ ]:
dls = generate_tsai_dataset(train_signals, train_labels)

## Select model

In [ ]:
######################################################
### models trained on normalized version of RAW TS ###
######################################################

# creating InceptionTime (is a CNN) model (vars: 5 (5 channels), c: 2 (2 classes))
# model = InceptionTime(dls.vars, dls.c)

# TODO: InceptionTimePlus

# TODO: XceptionTime

# TODO: XceptionTimePlus

# TODO: OmniScaleCNN

# creating XCM
# eXplainable Convolutional neural network for Multivariate time series classification (XCM)
# model = XCM(dls.vars, dls.c, dls.len)

# creating XCMPlus
# eXplainable Convolutional neural network for Multivariate time series classification (XCM)
model = XCMPlus(dls.vars, dls.c, dls.len)

# creating FCN (CNN model)
# model = FCN(dls.vars, dls.c)

# TODO: FCNPlus

# creating ResNet (CNN)
# model = ResNet(dls.vars, dls.c)

# TODO: ResNetPlus

# TODO: XResNet1d

# TODO: XResNet1dPlus

# TODO: ResCNN

# TODO: TCN

# creating RNN
# model = RNN(dls.vars, dls.c)

# creating RNNPlus (RNN model + including a feature extractor to the RNN network)
# model = RNNPlus(dls.vars, dls.c)

# TODO: RNNAttention

# creating GRU (RNN model)
# model = GRU(dls.vars, dls.c)

# creating GRUPlus (RNN model + including a feature extractor to the RNN network)
# model = GRUPlus(dls.vars, dls.c)

# creating GRUAttention (RNN model + attention)
# model = GRUAttention(dls.vars, dls.c, seq_len=500)

# creating LSTM (RNN model)
# model = LSTM(dls.vars, dls.c)

# creating LSTMPlus (RNN model + including a feature extractor to the RNN network)
# model = LSTMPlus(dls.vars, dls.c)

# creating LSTMAttention (RNN model + attention)
# model = LSTMAttention(dls.vars, dls.c, seq_len=500)

# creating TSSequencerPlus
# model = TSSequencerPlus(dls.vars, dls.c, seq_len=500)

# creating TransformerModel
# model = TransformerModel(dls.vars, dls.c)

# TODO: TST

# TODO: TSTPlus

# TODO: TSPerceiver

# TODO: TSiT

# TODO: PatchTST

# TODO: ROCKETs category

# TODO: Wavelet-based NNs category

# TODO: Hybrid models category

# TODO: Tabular models category

#########################################
### models trained on feature vectors ###
#########################################

# TODO: extract + select features, i.e., generate feature vectors

# TODO: MLP

# TODO: gMLP

## Training

In [ ]:
# there should be no model, otherwise retraining!
assert not os.path.isdir("export")
assert not os.path.isdir("models")

learn = train_tsai_model(dls, model)

In [ ]:
# losses -> loss development over all epochs for 'train' and 'valid'
# final losses ->  zoomed-in view of the final epochs, focusing on loss values towards the end of training
# accuracy -> validation accuracy of the model
learn.recorder.plot_metrics()

In [ ]:
learn.save_all(path='export', dls_fname='dls', model_fname='model', learner_fname='learner')

In [ ]:
learn.show_results(nrows=3, ncols=3)

In [ ]:
learn.show_probas()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
# interested in cases where the model made incorrect predictions at least 3 times
confusions = interp.most_confused(min_val=3)
for actual_class, pred_class, times in confusions:
    print("pred:", pred_class)
    print("actual:", actual_class)
    print(times, "times")

## Inference on additional data

In [ ]:
test_acc = test_tsai_model(test_signals, test_labels, learn)
print("test accuracy:", test_acc)

## GradCAM for XCM and XCMPlus

In [ ]:
assert type(model) in [tsai.models.XCMPlus.XCMPlus, tsai.models.XCM.XCM]

xb, yb = dls.one_batch()

print(yb[0])
print(xb[0])

model.show_gradcam(xb[0], yb[0], figsize=(10, 3))

In [ ]:
# as the built-in gradcam method creates plots that are sometimes unreadable, it is better to
# visualize it with the methods from oscillogram_classification.cam

# determine number of signals to be used for saliency map gen
num_test_samples = 10

# convert test data to CUDA tensors
xb = torch.tensor(test_signals[:num_test_samples], dtype=torch.float32).to('cuda:0')
yb = torch.tensor(test_labels[:num_test_samples], dtype=torch.float32).to('cuda:0')

input_data, probabilities, targets, predictions = learn.get_X_preds(xb.cpu(), yb.cpu(), with_input=True)
predictions = predictions.strip('][').split(', ')

In [ ]:
# random_index = random.randint(0, len(xb) - 1)

all_attr_maps = {}

for idx in range(len(xb)):
    if type(model) == tsai.models.XCMPlus.XCMPlus:
        att_maps = get_attribution_map(
            model,
            [model.backbone.conv2dblock, model.backbone.conv1dblock],
            xb[idx],
            detach=True,
            apply_relu=True
        )
    else:  # XCM
        att_maps = get_attribution_map(
            model,
            [model.conv2dblock, model.conv1dblock],
            xb[idx],
            detach=True,
            apply_relu=True
        )
    att_maps[0] = (att_maps[0] - att_maps[0].min()) / (att_maps[0].max() - att_maps[0].min())
    att_maps[1] = (att_maps[1] - att_maps[1].min()) / (att_maps[1].max() - att_maps[1].min())

    all_attr_maps[idx] = att_maps
    print("Ground truth: ", int(yb[idx]), " Prediction: ", predictions[idx])

plot_heatmaps_as_overlay(
    {"var. attr. map " + str(i) + "(gt: " + str(int(yb[i])) + ", pr: " + predictions[i] + ")": all_attr_maps[i][0].cpu().numpy()[0] for i in range(len(xb))},
    xb.cpu().numpy()[0].flatten(),
    'test_plot',
    np.array(range(len(xb[0].cpu().numpy()[0])))
)
plot_heatmaps_as_overlay(
    {"time attr. map " + str(i): all_attr_maps[i][1].cpu().numpy()[0] for i in range(len(xb))},
    xb.cpu().numpy()[0].flatten(),
    'test_plot',
    np.array(range(len(xb[0].cpu().numpy()[0])))
)

## Cross-validation for tsai training

In [ ]:
k = 10
train_test_splits = get_splits(
    np.concatenate((train_labels, test_labels), axis=0),
    n_splits=k,
    valid_size=.2,
    stratify=True,
    random_state=23,
    shuffle=True
)

In [ ]:
all_signals = np.concatenate((train_signals, test_signals), axis=0)
all_labels = np.concatenate((train_labels, test_labels), axis=0)
test_accuracies = []

for train_test_split in train_test_splits:
    train_split_signals = all_signals[train_test_split[0]]
    test_split_signals = all_signals[train_test_split[1]]
    train_split_labels = all_labels[train_test_split[0]]
    test_split_labels = all_labels[train_test_split[1]]

    # the training data will be further split into train and validation
    dls = generate_tsai_dataset(train_split_signals, train_split_labels)

    model = TransformerModel(dls.vars, dls.c)
    learn = train_tsai_model(dls, model)
    learn.save_all(path='export', dls_fname='dls', model_fname='model', learner_fname='learner')
    test_acc = test_tsai_model(test_split_signals, test_split_labels, learn)
    test_accuracies.append(test_acc)

In [ ]:
print(test_accuracies)
print("Mean accuracy over all folds: ", np.mean(test_accuracies))

## Load and apply already trained torch model

In [ ]:
import torch

model_path = "../data/Lambdasonde.pth"
model = torch.load(model_path)
# ensure model is in evaluation mode
model.eval()

tensors = torch.from_numpy(test_signals).float()

# iterate over test signals
for idx in range(tensors.shape[0]):
    # assumes model outputs logits for a multi-class classification problem
    logits = model(tensors[[idx]])
    # convert logits to probabilities using softmax
    probabilities = torch.softmax(logits, dim=1)
    print(probabilities)
    first_class = float(probabilities[0][0])
    second_class = float(probabilities[0][1])

    if first_class < second_class:
        print("pred POS \t ground truth:", test_titles[idx])
    else:
        print("pred NEG \t ground truth:", test_titles[idx])